In [1]:
from __future__ import print_function, division
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.nn import init
import torch.nn.functional as F
import functools
import numpy as np
import torchvision
from torchvision import transforms, datasets, models
import os
import time

import matplotlib.pyplot as plt

In [3]:
# Training settings
batch_size = 10000

In [27]:
#%%

# p = Augmentor.Pipeline()
# p.random_erasing(0.5,0.4)




train_transform = transforms.Compose([
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomCrop((32, 32), padding=4),   #left, top, right, bottom
#     p.torch_transform(),
#     p.sample(10000),
    transforms.ToTensor()
])
test_transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = datasets.CIFAR10(root='/media/qing/linux/repo/Assistance_node/Assistant_Unit/data',
                               train=True,
                               transform=train_transform,
                               download=False)

test_dataset = datasets.CIFAR10(root='/media/qing/linux/repo/Assistance_node/Assistant_Unit/data',
                              train=False,
                              transform=test_transform)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, # 64
                                           shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

dataloaders = {
    'train': train_loader,
    'test' : test_loader    
}
image_datasets = {
    'train': train_dataset,
    'test' : test_dataset
}
dataset_sizes = {
    'train': len(image_datasets['train']),
    'test' : len(image_datasets['test'])
}

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


print('train_size: ',dataset_sizes['train'],'\ntest_size : ',dataset_sizes['test'])
print(classes)
print('len class: ', len(classes))

train_size:  50000 
test_size :  10000
('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
len class:  10


In [28]:
for n, class_n in enumerate(classes):
    a = class_n
    print(n, a)

0 plane
1 car
2 bird
3 cat
4 deer
5 dog
6 frog
7 horse
8 ship
9 truck


In [38]:

import cv2
import random
# # ADDING NOISE
noise = []
noise = np.random.rand(32, 32,3)
import pickle

# pickle_out = open("batch1","wb")
data = []
filenames = []
occludes = []

for batch_n in range(5):
    inputs, labels = next(iter(dataloaders['train']))
#     print(inputs.shape)
    pickle_out = open("batch{}".format(batch_n+1),"wb")
    data = []
    count = [0 for i in range(10)]
    for m in range(batch_size):
        images_ = inputs[m].numpy()
        class_  = labels[m].numpy()
        images_ = np.transpose(images_, [1,2,0])
        for n, class_n in enumerate(classes):
            if classes[class_] == class_n:
                count[class_] += 1
                if count[class_] < 200:
                    x_random = np.random.randint(6, 27, size=1, dtype='uint8')
                    y_random = np.random.randint(6, 27, size=1, dtype='uint8')
                    w_random = np.random.randint(8, 13, size=1, dtype='uint8')
                    h_random = np.random.randint(8, 13, size=1, dtype='uint8')
                    images_[y_random[0]-int(h_random[0]/2):y_random[0]+int(h_random[0]/2),x_random[0]-int(w_random[0]/2):x_random[0]+int(w_random[0]/2),:]=noise[y_random[0]-int(h_random[0]/2):y_random[0]+int(h_random[0]/2),x_random[0]-int(w_random[0]/2):x_random[0]+int(w_random[0]/2),:]

                    r = images_[:,:,0]
                    g = images_[:,:,1]
                    b = images_[:,:,2]

                    images_[:,:,0] = (r - r.min()) / (r.max() - r.min())
                    images_[:,:,1] = (g - g.min()) / (g.max() - g.min())
                    images_[:,:,2] = (b - b.min()) / (b.max() - b.min())
            #             encode_img = images_.tostring()
                    data.append(images_)
                    filename = '{}_{}_{}.png'.format(classes[class_],class_,batch_n*10000+m)
                    filenames.append(filename)
                    occlude = 1
                    occludes.append(occlude)

#                     plt.imsave('/home/lee/Research/11.project_tangqing/data/cifar-10-occlude/{}_{}.png'.format(classes[class_],class_*10000+m), images_)

                else:
                    r = images_[:,:,0]
                    g = images_[:,:,1]
                    b = images_[:,:,2]

                    images_[:,:,0] = (r - r.min()) / (r.max() - r.min())
                    images_[:,:,1] = (g - g.min()) / (g.max() - g.min())
                    images_[:,:,2] = (b - b.min()) / (b.max() - b.min())
            #             encode_img = images_.tostring()
                    data.append(images_)
                    filename = '{}_{}_{}.png'.format(classes[class_],class_,batch_n*10000+m)
                    filenames.append(filename)
#         print(classes[class_.numpy()])
#                     plt.imsave('/home/lee/Research/11.project_tangqing/data/cifar-10-occlude/{}_{}.png'.format(classes[class_],class_*10000+m), images_)

                    occlude = 0
                    occludes.append(occlude)
#         plt.imshow(images_)
#         plt.show()
        
        
    pickle_cifar10 = {'labels': labels.numpy(),'data': data, 'filenames': filenames, 'occludes': occludes}
    pickle.dump(pickle_cifar10, pickle_out)
    pickle_out.close()
    del data[:]
    #data.clear()
    del filenames[:]
    del count[:]
    del occludes[:]

In [39]:
def unpickle(file):
#     import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo)
    return dict

In [40]:
pickle_in = unpickle("/media/qing/linux/repo/Assistance_node/Assistant_Unit/batch1")
# print(len(pickle_in['occludes']))
# print(pickle_in['occludes'])

# print(a[100])
#print(pickle_in['data'][0])
print(pickle_in['labels'])
print(pickle_in['filenames'])

[6 9 9 ... 1 1 5]
['frog_6_0.png', 'truck_9_1.png', 'truck_9_2.png', 'deer_4_3.png', 'car_1_4.png', 'car_1_5.png', 'bird_2_6.png', 'horse_7_7.png', 'ship_8_8.png', 'cat_3_9.png', 'deer_4_10.png', 'horse_7_11.png', 'horse_7_12.png', 'bird_2_13.png', 'truck_9_14.png', 'truck_9_15.png', 'truck_9_16.png', 'cat_3_17.png', 'bird_2_18.png', 'frog_6_19.png', 'deer_4_20.png', 'cat_3_21.png', 'frog_6_22.png', 'frog_6_23.png', 'bird_2_24.png', 'frog_6_25.png', 'cat_3_26.png', 'dog_5_27.png', 'deer_4_28.png', 'plane_0_29.png', 'plane_0_30.png', 'truck_9_31.png', 'car_1_32.png', 'cat_3_33.png', 'deer_4_34.png', 'plane_0_35.png', 'cat_3_36.png', 'horse_7_37.png', 'cat_3_38.png', 'cat_3_39.png', 'dog_5_40.png', 'bird_2_41.png', 'bird_2_42.png', 'horse_7_43.png', 'car_1_44.png', 'car_1_45.png', 'car_1_46.png', 'bird_2_47.png', 'bird_2_48.png', 'plane_0_49.png', 'truck_9_50.png', 'dog_5_51.png', 'horse_7_52.png', 'truck_9_53.png', 'bird_2_54.png', 'bird_2_55.png', 'dog_5_56.png', 'bird_2_57.png', 'deer

In [25]:
print(len(pickle_in['data'][0]))
#print(pickle_in['labels'])
#print(pickle_in['filenames'])

32


In [45]:
# test dataset

# pickle_out = open("batch1","wb")
data = []
filenames = []
occludes = []

for batch_n in range(1):
    inputs, labels = next(iter(dataloaders['test']))
#     print(inputs.shape)
    pickle_out = open("batch{}".format(batch_n+1),"wb")
    data = []
    count = [0 for i in range(10)]
    for m in range(batch_size):
        images_ = inputs[m].numpy()
        class_  = labels[m].numpy()
        images_ = np.transpose(images_, [1,2,0])
        for n, class_n in enumerate(classes):
            if classes[class_] == class_n:
                count[class_] += 1
                if count[class_] < 200:
                    x_random = np.random.randint(6, 27, size=1, dtype='uint8')
                    y_random = np.random.randint(6, 27, size=1, dtype='uint8')
                    w_random = np.random.randint(8, 13, size=1, dtype='uint8')
                    h_random = np.random.randint(8, 13, size=1, dtype='uint8')
                    images_[y_random[0]-int(h_random[0]/2):y_random[0]+int(h_random[0]/2),x_random[0]-int(w_random[0]/2):x_random[0]+int(w_random[0]/2),:]=noise[y_random[0]-int(h_random[0]/2):y_random[0]+int(h_random[0]/2),x_random[0]-int(w_random[0]/2):x_random[0]+int(w_random[0]/2),:]

                    r = images_[:,:,0]
                    g = images_[:,:,1]
                    b = images_[:,:,2]

                    images_[:,:,0] = (r - r.min()) / (r.max() - r.min())
                    images_[:,:,1] = (g - g.min()) / (g.max() - g.min())
                    images_[:,:,2] = (b - b.min()) / (b.max() - b.min())
            #             encode_img = images_.tostring()
                    data.append(images_)
                    filename = 'test_{}_{}_{}.png'.format(classes[class_],class_,batch_n*10000+m)
                    filenames.append(filename)
                    occlude = 1
                    occludes.append(occlude)

#                     plt.imsave('/home/lee/Research/11.project_tangqing/data/cifar-10-occlude/{}_{}.png'.format(classes[class_],class_*10000+m), images_)

                else:
                    r = images_[:,:,0]
                    g = images_[:,:,1]
                    b = images_[:,:,2]

                    images_[:,:,0] = (r - r.min()) / (r.max() - r.min())
                    images_[:,:,1] = (g - g.min()) / (g.max() - g.min())
                    images_[:,:,2] = (b - b.min()) / (b.max() - b.min())
            #             encode_img = images_.tostring()
                    data.append(images_)
                    filename = 'test_{}_{}_{}.png'.format(classes[class_],class_,batch_n*10000+m)
                    filenames.append(filename)
#         print(classes[class_.numpy()])
#                     plt.imsave('/home/lee/Research/11.project_tangqing/data/cifar-10-occlude/{}_{}.png'.format(classes[class_],class_*10000+m), images_)

                    occlude = 0
                    occludes.append(occlude)
#         plt.imshow(images_)
#         plt.show()
        
        
    pickle_cifar10 = {'labels': labels.numpy(),'data': data, 'filenames': filenames, 'occludes': occludes}
    pickle.dump(pickle_cifar10, pickle_out)
    pickle_out.close()
    del data[:]
    #data.clear()
    del filenames[:]
    del count[:]
    del occludes[:]

In [46]:
pickle_in = unpickle("/media/qing/linux/repo/Assistance_node/Assistant_Unit/batch1")
# print(len(pickle_in['occludes']))
# print(pickle_in['occludes'])

# print(a[100])
#print(pickle_in['data'][0])
print(pickle_in['labels'])
print(pickle_in['filenames'])

[3 8 8 ... 5 1 7]
['test_cat_3_0.png', 'test_ship_8_1.png', 'test_ship_8_2.png', 'test_plane_0_3.png', 'test_frog_6_4.png', 'test_frog_6_5.png', 'test_car_1_6.png', 'test_frog_6_7.png', 'test_cat_3_8.png', 'test_car_1_9.png', 'test_plane_0_10.png', 'test_truck_9_11.png', 'test_dog_5_12.png', 'test_horse_7_13.png', 'test_truck_9_14.png', 'test_ship_8_15.png', 'test_dog_5_16.png', 'test_horse_7_17.png', 'test_ship_8_18.png', 'test_frog_6_19.png', 'test_horse_7_20.png', 'test_plane_0_21.png', 'test_deer_4_22.png', 'test_truck_9_23.png', 'test_dog_5_24.png', 'test_bird_2_25.png', 'test_deer_4_26.png', 'test_plane_0_27.png', 'test_truck_9_28.png', 'test_frog_6_29.png', 'test_frog_6_30.png', 'test_dog_5_31.png', 'test_deer_4_32.png', 'test_dog_5_33.png', 'test_truck_9_34.png', 'test_bird_2_35.png', 'test_deer_4_36.png', 'test_car_1_37.png', 'test_truck_9_38.png', 'test_dog_5_39.png', 'test_deer_4_40.png', 'test_frog_6_41.png', 'test_dog_5_42.png', 'test_frog_6_43.png', 'test_plane_0_44.png',